# Task1: W(U)PGMA

In [1]:
import numpy as np
from collections import namedtuple

In [2]:
inf = np.inf
Node = namedtuple('Node', ('left', 'right', 'distance', 'size'))
Leaf = namedtuple('Leaf', ('name', 'size', 'distance'))

Cluster = namedtuple('Cluster', ('left', 'right', 'distance'))

In [3]:
def wpgma_step(m, verts, weighted=True):
    n = len(verts)
    
    # find min distance
    flat_index = np.argmin(m)
    i, j = flat_index // n, flat_index % n
    distance = m[i, j]
    
    # recalculate distances
    if weighted:
        ar = []
        for k in range(n):
            if k not in (i, j):
                ar.append([(m[min(k, i), max(k, i)] + m[min(k, j), max(k, j)]) / 2])
    else:
        weight_i = verts[i].size
        weight_j = verts[j].size
        ar = [[(m[min(v, i), max(v, i)] * weight_i + m[min(v, j), max(v, j)] * weight_j) / (weight_i + weight_j)] 
              for v in range(n) if v not in (i, j)]
    
    if n > 2:
        # remove distances to clustered objects
        for axis in (0, 1):
            m = np.delete(m, (j, i), axis=axis)

        # add distances to the newly formed cluster
        m = np.append(m, ar, axis=1)
        m = np.append(m, [[inf for _ in range(n - 1)]] , axis=0)
       
    else: 
        m = None
        
    a = verts.pop(i)
    b = verts.pop(j - 1)
    verts.append(Node(a, b, distance, a.size + b.size))
    
    return m, verts
    

In [4]:
def wpgma_to_newick(tree):
    if tree.size == 1:
        return tree.name, 0
    else:
        left, l_dist = wpgma_to_newick(tree.left)
        right, r_dist = wpgma_to_newick(tree.right)
        
        return f'({left}:{tree.distance / 2 - l_dist},{right}:{tree.distance / 2 - r_dist})', tree.distance / 2
    

In [5]:
def wpgma(m, leafs, weighted=True):
    verts = [Leaf(v, 1, 0) for v in leafs]
    
    while m is not None:
        m, verts = wpgma_step(m, verts, weighted)
    
    newick, _ = wpgma_to_newick(verts[0])
    return newick
    

# Task2: NJ

In [6]:
def nj_step(m, verts):
    n = len(verts)
    
    if n == 2:
        distance = m[0, 1]
        return None, [Cluster(verts[0], verts[1], (distance / 2, distance / 2))]
    
    def mean_dist(i, j):
        dist = 0
        for k in range(n):
            if k not in (i, j):
                a = m[min(k, i), max(k, i)]
                dist += a if a != inf else 0
        return dist / (n - 2)
    
    d = np.empty((n, n), dtype=np.float32)
    d.fill(inf)
    
    for i in range(n):
        for j in range(i, n):
            dist = m[i, j]
            d[i, j] = dist - mean_dist(i, j) - mean_dist(j, i)
            
    # find min distance
    flat_index = np.argmin(d)
    i, j = flat_index // n, flat_index % n
    distance = m[i, j]
    diff = mean_dist(i, j) - mean_dist(j, i) 
    
    # recalculate distances
    ar = []
    for k in range(n):
        if k not in (i, j):
            ar.append([(m[min(k, i), max(k, i)] + m[min(k, j), max(k, j)] - distance) / 2])
    

    # remove distances to clustered objects
    for axis in (0, 1):
        m = np.delete(m, (j, i), axis=axis)

    # add distances to the newly formed cluster
    m = np.append(m, ar, axis=1)
    m = np.append(m, [[inf for _ in range(n - 1)]] , axis=0)
       
    a = verts.pop(i)
    b = verts.pop(j - 1)
    verts.append(Cluster(a, b, ((distance + diff) / 2, (distance - diff) / 2)))
    
    return m, verts

In [7]:
def nj_to_newick(tree):
    if tree.distance is None:
        return tree.name
    else:
        left = nj_to_newick(tree.left)
        right = nj_to_newick(tree.right)
        l_dist, r_dist = tree.distance
        
        return f'({left}:{l_dist},{right}:{r_dist})'

In [8]:
def nj(m, leafs):
    verts = [Leaf(v, 1, None) for v in leafs]
    
    while m is not None:
        m, verts = nj_step(m, verts)

    newick = nj_to_newick(verts[0])
    return newick

## Test 1

In [9]:
leafs = ['A', 'B', 'C', 'D']
distances = np.array([[inf,  16,  16,  10],
                      [inf, inf,   8,   8],
                      [inf, inf, inf,   4],
                      [inf, inf, inf, inf]], dtype=np.float32)

### WPGMA

In [10]:
w = wpgma(distances, leafs)
w

'(A:7.25,(B:4.0,(C:2.0,D:2.0):2.0):3.25)'

### UPGMA

In [11]:
w = wpgma(distances, leafs, weighted=False)
w

'(A:7.0,(B:4.0,(C:2.0,D:2.0):2.0):3.0)'

### NJ

In [12]:
w = nj(distances, leafs)
w

'(A:5.25,(B:5.5,(C:3.5,D:0.5):0.5):5.25)'

## Test 2

In [13]:
leafs2 = ['A', 'B', 'C', 'D', 'E', 'F']
distances2 = np.array([[inf,   5,   4,   7,   6,   8],
                       [inf, inf,   7,  10,   9,  11],
                       [inf, inf, inf,   7,   6,   8],
                       [inf, inf, inf, inf,   5,   9],
                       [inf, inf, inf, inf, inf,   8],
                       [inf, inf, inf, inf, inf, inf]], dtype=np.float32)

### WPGMA

In [14]:
w2 = wpgma(distances2, leafs2)
w2

'(F:4.5,((D:2.5,E:2.5):1.5,(B:3.0,(A:2.0,C:2.0):1.0):1.0):0.5)'

### UPGMA

In [15]:
w2 = wpgma(distances2, leafs2, weighted=False)
w2

'(F:4.4,((D:2.5,E:2.5):1.25,(B:3.0,(A:2.0,C:2.0):1.0):0.75):0.6500000000000004)'

### NJ

In [16]:
w2 = nj(distances2, leafs2)
w2

'(F:2.5,((C:2.0,(A:1.0,B:4.0):1.0):1.0,(D:3.0,E:2.0):1.0):2.5)'